# UEC実践ソフトウェア開発基礎論2023-レポート課題2
田中久温

## 概要
元コンペ: https://www.kaggle.com/competitions/bnp-paribas-cardif-claims-management \
個人保険のデータセット．顧客から来た保険金の請求を速やかにチェックして承認 or 非承認しないといけない．\
データセットに含まれる請求には2つのカテゴリが含まれていて，どっちのカテゴリか判定したい．\
出力は確率，評価指標は logloss \
目的変数も説明変数も匿名で，特に説明無し．順序変数はないらしい．

## 手法
この notebook では，XGBoost を試す．\
この notebook を最初に実装し，これをベースに他の手法も試す．

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv("../input/bnp-paribas-cardif-claims-management/train.csv.zip")
test_df = pd.read_csv("../input/bnp-paribas-cardif-claims-management/test.csv.zip")
sample_df = pd.read_csv("../input/bnp-paribas-cardif-claims-management/sample_submission.csv.zip")

# EDA
データを見てみる．

In [3]:
train_df

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114316,228708,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AL,NaN,NaN,NaN,0,NaN,NaN
114317,228710,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,E,NaN,NaN,NaN,1,NaN,NaN
114318,228711,1,NaN,NaN,C,NaN,10.069277,NaN,NaN,0.323324,...,NaN,NaN,0.156764,Q,NaN,NaN,2.417606,2,NaN,NaN
114319,228712,1,NaN,NaN,C,NaN,10.106144,NaN,NaN,0.309226,...,NaN,NaN,0.490658,BW,NaN,NaN,3.526650,0,NaN,NaN


In [4]:
test_df

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,0,1.375465e+00,11.361141,C,4.200778,6.577000,2.081784,1.784386,0.011094,9.523810,...,7.619048,1.815241,1.112270e-07,AF,1.292368,3.903345,1.485925,0,2.333334,1.428572e+00
1,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,I,NaN,NaN,NaN,0,NaN,NaN
2,2,-4.903407e-07,8.201529,C,4.544371,6.550100,1.558442,2.467532,0.007164,7.142858,...,5.714286,1.970928,1.412265e-02,AV,1.128724,5.844156,1.475892,0,1.263157,-6.380022e-07
3,7,2.661870e+00,3.041241,C,1.657216,9.773080,2.078337,1.430855,1.252157,7.959596,...,4.404040,8.163614,1.100329e+00,B,1.988688,1.558753,2.448814,0,5.385474,1.493777e+00
4,10,1.252822e+00,11.283352,C,4.638388,8.520510,2.302484,3.510159,0.074263,7.612904,...,6.580644,1.325654,2.584588e-01,A,1.863796,2.666478,2.374275,0,0.681672,2.264151e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114388,228700,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AG,NaN,NaN,NaN,0,NaN,NaN
114389,228703,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,H,NaN,NaN,NaN,0,NaN,NaN
114390,228704,1.523740e+00,6.571597,C,3.827086,8.824122,2.635660,2.218992,6.534959,8.852942,...,6.852941,2.093560,5.111015e+00,V,1.826453,2.661700,1.601567,0,1.816594,1.634616e+00
114391,228706,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AZ,NaN,NaN,NaN,0,NaN,NaN


めちゃくちゃいっぱい特徴量ある……

In [5]:
train_test_df = pd.concat([train_df, test_df])

In [6]:
describes = train_test_df.describe()
describes.iloc[:, :15]

,ID,target,v1,v2,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14
count,228714.000000,114321.000000,1.290030e+05,1.290790e+05,1.290790e+05,1.314480e+05,1.290030e+05,1.290030e+05,1.314590e+05,1.289770e+05,2.285480e+05,1.289970e+05,2.285440e+05,1.290030e+05,2.287080e+05
mean,114356.500000,0.761199,1.629654e+00,7.450246e+00,4.143181e+00,8.741077e+00,2.436143e+00,2.483027e+00,1.504411e+00,9.031849e+00,1.882734e+00,1.544710e+01,6.880435e+00,3.803983e+00,1.209657e+01
std,66024.189071,0.426353,1.077848e+00,2.952263e+00,1.148699e+00,2.043668e+00,5.934825e-01,5.909741e-01,2.793558e+00,1.926892e+00,1.395887e+00,7.879657e-01,9.235194e-01,1.182710e+00,1.443268e+00
min,0.000000,0.000000,-9.996497e-07,-9.817614e-07,-6.475929e-07,-5.287068e-07,-9.873300e-07,-9.468765e-07,-7.783778e-07,-9.828757e-07,-9.875317e-07,-1.459062e-07,5.143224e-07,-8.464889e-07,-9.738831e-07
25%,57178.250000,1.000000,9.147091e-01,5.309706e+00,3.486229e+00,7.593565e+00,2.064515e+00,2.098648e+00,8.628193e-02,7.849463e+00,1.050328e+00,1.500000e+01,6.322207e+00,3.067134e+00,1.125602e+01
50%,114356.500000,1.000000,1.465025e+00,7.002494e+00,4.204843e+00,8.660546e+00,2.411422e+00,2.451395e+00,3.846458e-01,9.065933e+00,1.312910e+00,1.549356e+01,6.612513e+00,3.596197e+00,1.196783e+01
75%,171534.750000,1.000000,2.134615e+00,9.443252e+00,4.831004e+00,9.778076e+00,2.775444e+00,2.833012e+00,1.632873e+00,1.023529e+01,2.100657e+00,1.595190e+01,7.015984e+00,4.294897e+00,1.272114e+01
max,228713.000000,1.000000,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01


特徴量多いし，これだけ見ても分かることあんまりないなぁ．

最小値の最小値だけ見ておく．（極端にマイナスに触れてる値があって，nan を-999とかで埋めたときに通常の値と混ざると嫌なので）

In [7]:
describes.iloc[3].describe()

count    1.140000e+02
mean    -6.346373e-07
std      5.006241e-07
min     -9.999976e-07
25%     -9.917094e-07
50%     -9.078924e-07
75%     -3.309738e-07
max      8.197812e-07
Name: min, dtype: float64

In [8]:
categorical_columns = [c for c in train_test_df.columns if train_test_df[c].dtype == 'O'] # 先生のサンプルから
categorical_columns

['v3',
 'v22',
 'v24',
 'v30',
 'v31',
 'v47',
 'v52',
 'v56',
 'v66',
 'v71',
 'v74',
 'v75',
 'v79',
 'v91',
 'v107',
 'v110',
 'v112',
 'v113',
 'v125']

In [9]:
for c in categorical_columns:
    print(c)
    counts = pd.value_counts(train_test_df[c], dropna=False)
    print(counts)
    if np.nan in counts.keys():
        print(f'nan: {counts[np.nan]}')
    else:
        print('nan: 0')
    print('-----------------------------')

v3
C      221224
NaN      6949
A         433
B         108
Name: v3, dtype: int64
nan: 6949
-----------------------------
v22
AGDF    4777
YGJ     4344
QKI     1302
PWR     1296
NaN      977
        ... 
BCT        1
FUD        1
IGO        1
RQO        1
VQW        1
Name: v22, Length: 23420, dtype: int64
nan: 977
-----------------------------
v24
E    110356
D     52689
C     41759
B     16318
A      7592
Name: v24, dtype: int64
nan: 0
-----------------------------
v30
NaN    120452
C       64504
G       17243
D       10342
E        6072
F        5160
A        4524
B         417
Name: v30, dtype: int64
nan: 120452
-----------------------------
v31
A      176937
B       37847
C        6981
NaN      6949
Name: v31, dtype: int64
nan: 6949
-----------------------------
v47
C    110853
I     78241
E     10652
F      8550
G      7829
D      6334
J      6078
B       111
A        65
H         1
Name: v47, dtype: int64
nan: 0
-----------------------------
v52
J      22452
I      20420
D      

nan が 5 のやつとかは，最頻値で埋めてもいいかも．

In [10]:
numeric_columns = [c for c in train_test_df.columns[2:] if c not in categorical_columns]
# numeric_columns

# 特徴量作成
とりあえず雑にやってみる．

## 欠損値処理

In [11]:
# numeric
for c in numeric_columns:
    nan_ratio = train_test_df[c].isna().mean()

    if nan_ratio < 0.1:
        # nan が 10% 未満なら平均値で埋める．(あんまり考慮したくない)
        column_mean = train_test_df[c].mean()
        train_test_df[c] = train_test_df[c].fillna(column_mean)
    else:
        # nan が 10% 以上あれば -999 で埋める．(モデルが考慮できるようにする)
        train_test_df[c] = train_test_df[c].fillna(-999)
    
# categorical
for c in categorical_columns:
    nan_count = train_test_df[c].isna().sum()

    if nan_count <= 10:
        # nan が 10個以下なら，最頻値で埋める．(あんまり考慮したくない)
        train_test_df[c] = train_test_df[c].fillna(train_test_df[c].mode()[0])
    else:
        # nan が 10個より多いなら，カテゴリとして扱う．(モデルが考慮できるようにする)
        train_test_df[c] = train_test_df[c].fillna(train_test_df[c].mode()[0])

## カテゴリ変数の変換

In [12]:
from sklearn.preprocessing import LabelEncoder

for c in categorical_columns:
    le = LabelEncoder()
    train_test_df[c] = le.fit_transform(train_test_df[c])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
train_test_df

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1.0,1.335739,8.727474,2,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,3.575369e-02,21,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1.0,-999.000000,-999.000000,2,-999.000000,9.191265,-999.000000,-999.000000,2.301630,...,-999.000000,-999.000000,5.988956e-01,6,-999.000000,-999.000000,1.957825,0,-999.000000,-999.000000
2,5,1.0,0.943877,5.310079,2,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,1.345191e-02,5,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1.0,0.797415,8.304757,2,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,2.267384e-03,64,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1.0,-999.000000,-999.000000,2,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,...,-999.000000,-999.000000,-9.990000e+02,89,-999.000000,-999.000000,-999.000000,0,-999.000000,-999.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114388,228700,NaN,-999.000000,-999.000000,2,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,...,-999.000000,-999.000000,-9.990000e+02,7,-999.000000,-999.000000,-999.000000,0,-999.000000,-999.000000
114389,228703,NaN,-999.000000,-999.000000,2,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,...,-999.000000,-999.000000,-9.990000e+02,71,-999.000000,-999.000000,-999.000000,0,-999.000000,-999.000000
114390,228704,NaN,1.523740,6.571597,2,3.827086,8.824122,2.635660,2.218992,6.534959,...,6.852941,2.093560,5.111015e+00,85,1.826453,2.661700,1.601567,0,1.816594,1.634616
114391,228706,NaN,-999.000000,-999.000000,2,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,...,-999.000000,-999.000000,-9.990000e+02,26,-999.000000,-999.000000,-999.000000,0,-999.000000,-999.000000


# とりあえず学習させてみる

In [14]:
use_cols = numeric_columns + categorical_columns

In [15]:
# データ分割
from sklearn.model_selection import train_test_split 

train_ft_df = train_test_df[train_test_df['target'].notna()][use_cols]
train_y_df = train_test_df[train_test_df['target'].notna()]['target'].astype(int)
test_ft_df = train_test_df[train_test_df['target'].isna()][use_cols]

train_X, valid_X, train_y, valid_y = train_test_split(train_ft_df, train_y_df, test_size=0.2, shuffle=True)

In [16]:
import xgboost as xgb


In [17]:
dtrain = xgb.DMatrix(train_X, label=train_y)
dvalid = xgb.DMatrix(valid_X, label=valid_y)

In [18]:
n_rounds = 1000
early_stopping_rounds = 5
clf = xgb.train({'objective': 'binary:logistic'}, dtrain, n_rounds, evals=[(dtrain, 'train'), (dvalid, 'eval')], early_stopping_rounds=early_stopping_rounds)

[0]	train-logloss:0.59632	eval-logloss:0.59673
[1]	train-logloss:0.54449	eval-logloss:0.54539
[2]	train-logloss:0.51454	eval-logloss:0.51643
[3]	train-logloss:0.49630	eval-logloss:0.49900
[4]	train-logloss:0.48482	eval-logloss:0.48842
[5]	train-logloss:0.47756	eval-logloss:0.48215
[6]	train-logloss:0.47271	eval-logloss:0.47822
[7]	train-logloss:0.46917	eval-logloss:0.47590
[8]	train-logloss:0.46642	eval-logloss:0.47442
[9]	train-logloss:0.46430	eval-logloss:0.47383
[10]	train-logloss:0.46238	eval-logloss:0.47295
[11]	train-logloss:0.46060	eval-logloss:0.47244
[12]	train-logloss:0.45858	eval-logloss:0.47135
[13]	train-logloss:0.45681	eval-logloss:0.47085
[14]	train-logloss:0.45548	eval-logloss:0.47064
[15]	train-logloss:0.45411	eval-logloss:0.47018
[16]	train-logloss:0.45264	eval-logloss:0.47003
[17]	train-logloss:0.45104	eval-logloss:0.47005
[18]	train-logloss:0.45017	eval-logloss:0.46997
[19]	train-logloss:0.44960	eval-logloss:0.46977
[20]	train-logloss:0.44805	eval-logloss:0.47006
[2

## とりあえず submit してみる

In [19]:
pred_test = clf.predict(xgb.DMatrix(test_ft_df))

In [20]:
sub_df = pd.DataFrame({
    "ID": test_df["ID"], 
    "PredictedProb": pred_test
}).to_csv("tanaka_sub_xgb.csv", index=False)

public score: 0.47179

(private の方がちょっとだけ低かったけど private 見ながらチューニングしても仕方ないので見なかったことにする)

もうちょい．改善の余地あり．

ハイパラ調整 → 他のモデル試す → 他のモデルの方が精度いいなら，特徴量向上の余地あり．

っていう流れでやる．

In [21]:
from sklearn.model_selection import GridSearchCV

grid search の方法参考
https://qiita.com/c60evaporator/items/a9a049c3469f6b4872c6#%E3%82%B0%E3%83%AA%E3%83%83%E3%83%89%E3%82%B5%E3%83%BC%E3%83%81%E3%81%AE%E5%A0%B4%E5%90%88

In [22]:
cv_params = {
    'learning_rate': [0.05, 0.1, 0.3],
    'min_child_weight': [1, 2],
    'max_depth': [2, 4, 6, 8],
    # subsample とかもやろうとしたらめちゃめちゃ時間かかったのでボツ
}

clf = xgb.XGBClassifier(
    n_estimators=1000,
    objective = 'binary:logistic',
    early_stopping_rounds=10,
    verbosity=0,
)

gridcv = GridSearchCV(
    clf, cv_params, cv=3,
    scoring='neg_log_loss', n_jobs=-1
)
gridcv.fit(
    train_X, train_y,
    eval_set = [(valid_X, valid_y)]
)

best_params = gridcv.best_params_
best_score = gridcv.best_score_
print(f'最適パラメータ {best_params}\nスコア {best_score}')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[0]	validation_0-logloss:0.67703
[0]	validation_0-logloss:0.67692
[0]	validation_0-logloss:0.67703
[0]	validation_0-logloss:0.67703
[1]	validation_0-logloss:0.66224
[1]	validation_0-logloss:0.66246
[1]	validation_0-logloss:0.66249
[1]	validation_0-logloss:0.66246
[2]	validation_0-logloss:0.64899
[2]	validation_0-logloss:0.64914
[2]	validation_0-logloss:0.64914
[2]	validation_0-logloss:0.64933
[3]	validation_0-logloss:0.63706
[3]	validation_0-logloss:0.63718
[3]	validation_0-logloss:0.63706
[3]	validation_0-logloss:0.63684
[4]	validation_0-logloss:0.62610
[4]	validation_0-logloss:0.62610
[4]	validation_0-logloss:0.62611
[4]	validation_0-logloss:0.62578
[5]	validation_0-logloss:0.61610
[5]	validation_0-logloss:0.61610
[5]	validation_0-logloss:0.61611
[5]	validation_0-logloss:0.61575
[6]	validation_0-logloss:0.60697
[6]	validation_0-logloss:0.60697
[6]	validation_0-logloss:0.60696
[6]	validation_0-logloss:0.60658
[7]	validation_0-logloss:0.59862
[7]	validation_0-logloss:0.59862
[7]	valida

最適パラメータ {'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 1}

スコア -0.4677854976602324

In [23]:
pred_test = gridcv.predict_proba(test_ft_df)[:, 1]
pred_test

array([0.21176931, 0.8808892 , 0.86519724, ..., 0.83997864, 0.9444008 ,
       0.53527325], dtype=float32)

In [24]:
sub_df = pd.DataFrame({
    "ID": test_df["ID"], 
    "PredictedProb": pred_test
}).to_csv("tanaka_sub_xgb.csv", index=False)

public 0.466 で良くなった．

一旦他の手法の score を見てからこれ以上チューニングするか考える．

CatBoost が 0.4489 で CatBoost の方が明らかに良い．

これ以上はカウントエンコーディングも入れるくらいしか，思いつかないので，CatBoost に勝てなさそう．

この課題はここで終了．